In [7]:
import pandas as pd
import os
import folium
import numpy as np



def calculate_weights(lat, lon, sources):
    distances = [np.sqrt((lat - src_lat)**2 + (lon - src_lon)**2) for src_lat, src_lon in sources]
    inverse_distances = [1 / d if d != 0 else 1e6 for d in distances]  # Avoid division by zero
    total = sum(inverse_distances)
    weights = [idw / total for idw in inverse_distances]
    return weights

# Function to blend colors based on weights
def blend_colors(weights, colors):
    blended = [0, 0, 0]
    for weight, color in zip(weights, colors):
        blended[0] += weight * color[0]
        blended[1] += weight * color[1]
        blended[2] += weight * color[2]
    return f"rgb({int(blended[0])}, {int(blended[1])}, {int(blended[2])})"


In [89]:
# Assign distinct colors to the source points
source_colors = [
    (255, 0, 0),      # Dark Red
    (0, 0, 255),      # Dark Blue
    (0, 255, 0),      # Dark Green
    (255, 0, 255),     # Dark Purple
    (0, 0, 0 ),  # Black
    (255, 255, 0),    # Dark Yellow
    (255, 255, 255),   # white

]


stations = []
# Load dfs 
root = "../data/external_data/dfs_final/"
df_dict = {}
for file in os.listdir(root):
    if file.endswith(".csv"):
        df = pd.read_csv(root + file, index_col=0)
        stations.append((df['latitude'].iloc[0], df['longitude'].iloc[0]))

center_lat = -19.89821009966734
center_lon = -43.961620690949324
min_lat = -20.04565938183644
max_lat = -19.765459819162825
min_lon = -44.09176621445223
max_lon = -43.83564744003997


In [92]:
# Create a Folium map centered at the mean location of the stations
mean_lat = center_lat
mean_lon = center_lon
m = folium.Map(location=[mean_lat, mean_lon], zoom_start=12)
title_html = '''
             <h3 align="center" style="font-size:20px"><b> Estações Meteorológicas e suas Zonas de Influência</b></h3>
             '''
m.get_root().html.add_child(folium.Element(title_html))
# Define the grid resolution (lat-lon steps)
lat_step = 0.001
lon_step = 0.001
square_size = 0.0005

opacity = 0.5
# Create the grid and calculate the weighted influence
for lat in np.arange(min_lat, max_lat, lat_step):
    for lon in np.arange(min_lon, max_lon, lon_step):
        weights = calculate_weights(lat, lon, stations)
        color = blend_colors(weights, source_colors)
        bounds = [
            [lat - square_size, lon - square_size],  # Bottom-left corner
            [lat + square_size, lon + square_size]   # Top-right corner
        ]
    
        folium.Rectangle(
            bounds=bounds,

            location=[lat, lon],
            color=color,  # Color range from green to red
            opacity=0,
            fill=True,
            fill_opacity=opacity,
            fill_color=color,
        ).add_to(m)

# Add the stations to the map
for station in stations:
    folium.Marker(location=station, popup='Station').add_to(m)


info_df = pd.read_csv('../data/final_data.csv')
armad_pos = info_df[['latitude', 'longitude']].drop_duplicates().reset_index(drop=True)


for nlat,nlong in armad_pos.values:
        folium.Circle(
            radius=1,
            location=[nlat, nlong],
            color='blue',  # Color range from green to red
            opacity=0.5,
            fill=True,
            fill_opacity=0.5,
            fill_color='blue',
        ).add_to(m)
# Save and display the map

m.save("../results/maps/inverse_distance_map.html")
